In [1]:
# Data
import pandas as pd
import numpy as np
import country_converter as coco

In [2]:
df = pd.read_csv('Data/data_science_jobs_dataset.csv')

In [3]:
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
78,2022,EN,FT,Data Analyst,50000,USD,50000,AR,100,AR,L
354,2022,SE,FT,Data Engineer,156600,USD,156600,US,100,US,M
265,2022,SE,FT,Data Scientist,260000,USD,260000,US,100,US,M


## Cleaning

### Abreviations

We'll change the abreviations to the actual meaning for extra clarity:
- Experience level:
    - EN: Junior
    - MI: Intermediate
    - SE: Senior
    - EX: Executive
- Employment:
    - FT: Full time
    - PT: Part time
    - CT: Contract
    - FL: Freelance
- Country:
    - Every ISO2 abreviation is changed to the short name of the country
- Remote:
    - 100: Fully Remote
    - 50: Partially Remote
    - 0: No remote work
- Company size:
    - S: Small (less than 50 employees)
    - M: Medium (between 50 and 150 employees)
    - L: Large (more than 150 employees)

In [4]:
experience_rename = {"EN":"Junior", "MI":"Intermediate", "SE":"Senior", "EX":"Executive"}
employment_rename = {"FT":"Full time", "PT":"Part time", "CT":"Contract", "FL":"Freelance"}
country_rename = coco.CountryConverter().get_correspondence_dict('ISO2', 'name_short')
country_rename["GB"] = "United Kingdom" #GB'S conversion is missing from the premade dictionary, so we add it manually
remote_rename = {100:"Fully remote", 50:"Partially remote", 0:"No remote work"}
size_rename = {"S":"Small", "M":"Medium", "L":"Large"}

In [5]:
df = df.replace({"experience_level": experience_rename, "employment_type":employment_rename, 
                 "employee_residence":country_rename, "remote_ratio":remote_rename, 
                 "company_location":country_rename, "company_size":size_rename})
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
589,2021,Intermediate,Full time,Applied Data Scientist,68000,CAD,54238,United Kingdom,Partially remote,Canada,Large
396,2022,Intermediate,Full time,Data Scientist,102100,USD,102100,United States,No remote work,United States,Medium
503,2022,Intermediate,Full time,Data Scientist,150000,PLN,34650,Poland,Fully remote,Poland,Large


### Droping columns

To be able to make comparisons, we are only interested in the salary in USD. We will drop the columns salary and salary_currency.

In [6]:
df = df.drop(["salary", "salary_currency"], axis=1)
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
294,2022,Senior,Full time,Data Scientist,120160,United States,Fully remote,United States,Medium
336,2022,Senior,Full time,Data Architect,90700,Canada,Fully remote,Canada,Medium
148,2022,Senior,Full time,Data Engineer,130000,United States,Fully remote,United States,Medium


In [7]:
df.to_csv("data/cleaned_data_science_jobs_dataset.csv", index=False)

# EDA

## Feature analysis univariate

### Work Year

<div>
<img src="Images/Work_year.png" width="400"/>
</div>

Every year there is a noticable increase in the number of data in our dataset. More available jobs related to data worldwide could explain a small difference, but the reason for an increase this big is that the data is entered voluntarily and the site that collects it has been gaining popularity over the last 2 years.

### Experience level

<div>
<img src="Images/Experience_level.png" width="700"/>
</div>

More than half of the positions are Senior, and aproximately one out of three are intermediate, one out of seven are junior and only one out of 25 are executive level positions

### Employment type

<div>
<img src="Images/Employment_type.png" width="400"/>
</div>

Almost every single job listed is a full time job.

### Job title

In [8]:
len(df["job_title"].unique())

52

<div>
<img src="Images/Job_title.png" width="900"/>
</div>

There are 52 unique job titles. Some of them are very niche, so we will focus on the 10 most common. The vast majority of data related employees are either Data Engineers, Data Scientists, Data Analysts or Machine Learning Engineers; over 80% of all job titles fall in these 4 categories.

### Salary (USD)

<div>
<img src="Images/Salary.png" width="500"/>
</div>

<div>
<img src="Images/Salary_2.png" width="1000"/>
</div>

The first graph shows that there is a large number of outliers in our dataset. The maximum salary is 900k\\$ per year! 

The second graph is the same boxplot as the first with the outliers filtered out, to make it easier to understand. There we can see that half of all the salaries are between 57k\\$ and 143k\\$ (first and third quartiles), with the median being at around 95k\\$.

The third graph, a bar plot of binned salaries (each bin having a range of 30k\\$) shows that the salaries follow a distribution close to normal, with some outliers and with an obvious lower bound of 0\\$. The mean of this distribution whould be in the 90k-120k\\$ bin, which will be checked in the next block of code:

In [9]:
mean = round(df["salary_in_usd"].mean(),2)
print(f"The mean of all salaries is {mean}$")

The mean of all salaries is 118856.37$


As expected, the mean falls in that bin. Lets check its value if we drop the outliers:

In [10]:
mean = round(df[(df["salary_in_usd"] < df["salary_in_usd"].quantile(0.75))]["salary_in_usd"].mean(),2)
print(f"The mean of salaries (excluding outliers) is {mean}$")

The mean of salaries (excluding outliers) is 86997.91$


### Employee residence

In [11]:
len(df["employee_residence"].unique())

62

<div>
<img src="Images/Employee_residence.png" width="900"/>
</div>

As with job titles, there are too many employee countries of residence to show in a graph (62), so only the 10 most common ones were plotted. The vast majority of our data (68.5%) comes from the United States.

### Remote Ratio

<div>
<img src="Images/Remote_work.png" width="500"/>
</div>

Interestingly, more than half of all the jobs are fully remote. Later we will see the evolution over time of this percentage, to see wether covid has had any impact on it.

### Company Location

<div>
<img src="Images/Company_location.png" width="900"/>
</div>

Similarly to employee residence, the country with the greater amount of jobs is by far the United States, with over 70% of all the share. The difference in percentage between company location and employee residence (1.5% in the case of USA) is explained by people working remotely from other countries. This phenomenon will be explored in more detail later.

### Company Size

<div>
<img src="Images/Company_size.png" width="700"/>
</div>

Medium companies (companies with more than 50 employees but less than 150 employees) make up more than half of all companies in the data.

## Feature analysis multivariate

### Job title by experience level

<div>
<img src="Images/Job_title_by_experience_level.png" width="900"/>
</div>

This bar plot is the same as the one shown before in the job titles section, but this time only the 8 most common job titles are shown, and each job is split by experience level (junior, mid, senior and executive). Inside each job title the percentage of each experience level (regarding that specific job title) is shown as text. 

The jobs with the biggest percentage of juniors are Research Scientists, Machine Learning Engineers, Data Scientists and Data Analysts.

The biggest percentage of intermediate level employees are in Research Science, Data Science and Data Engineering jobs

Seniors are the biggest percentage in Data Architecture and Data Science Managing jobs.

Executives occupy the biggest percentage as Analytics Engineers and Data Science Managers.

### Remote ratio by work year

### Remote Ratio by experience level

Compared to remote ratio overall

### Remote Ratio by employment type

Compared to remote ratio overall

### Remote Ratio by job title

Compared to remote ratio overall

### Remote Ratio by Country

Compared to remote ratio overall

### Remote Ratio by company size

Compared to remote ratio overall

### Remote Ratio when employee residence = company location

### Employee residence by Company location

To see how many remote jobs are from employees living in a different country

Useful if you want to look for remote jobs, so you know where to look

### Experience level by company size

### Salary by work year

### Salary by experience level

### Salary by job title

### Salary by job title by experience level

### Salary by job title by company location

### Salary by remote ratio

### Salary by company location

### Salary by company size

# Prediction